# ML Potentials Exercises July 19 
## Kernel Ridge Regression and Support Vector Regression


### Adapted from course: 2021W 270080-1 Machine learning for molecules and materials - Philipp Marquetand

In this exercise, we will learn how to train Kernel ridge regression (KRR) and Support vector machine (SVM) models. We will use the QM7 dataset. It contains more than 7000 different molecules with a maximum number of 23 atoms per molecule. We will try different types of kernels, vary their hyperparameters and use the Coulomb matrix as our chemical environment descriptor. Finally, we compare KRR and SVM.

### 1.1 Load libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import SVR

### 1.2 Import and inspect QM7 data
We will work with only 3500 molecules of the QM7 dataset due to time restrictions. The `qm7_files` directory consists of the `hof_qm7.txt` file and a `qm7` subdirectory with 7172 xyz-files. The `hof_qm7.txt` file contains the path to a molecule's xyz-file and its DFT/PBE0 and DFTB3 energy (heat of formation, hof). Each xyz-file represents one molecule. It contains information about the coordinates of the atoms, their chemical identity and their charges.

In [ ]:
#QM7 molecules
#We will include 3500 data points
n_molecules = 3500
data = pd.read_csv('qm7_files/hof_qm7.txt', skiprows=0, delim_whitespace=True,  names = ["input","PBE0","DFTB3"])
#print(data)
#print(data.input)

#Our X values will be the Coulomb matrices of the molecules,
##therefore we need to have a list of paths to the xyz-files of the molecules
inputfiles = []
for i in range(n_molecules):
    inputfile = "%s" %data.input[i] 
    inputfiles.append(inputfile)


#Our Y values are the PBE0 energies
Energy = np.zeros((n_molecules,1))
for i in range(n_molecules):
    Energy[i] = data.PBE0[i]

Let us have a look at some of these molecules.
If you click an atom in the viewer, the last digit in the readout will tell you what element it is. 

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
import nglview as nv
import MDAnalysis as mda

print("The first entry is methane: \nits atomization energy    = ", Energy[0], "kcal/mol")
mda_traj = mda.Universe(inputfiles[0])
mda_view = nv.show_mdanalysis(mda_traj)
mda_view

In [ ]:
#Change k to inspect some molecules by yourself!
k = 3485
print("Atomization energy of entry " + str(k) + " =", Energy[k], "kcal/mol")
mda_traj = mda.Universe(inputfiles[k])
mda_view = nv.show_mdanalysis(mda_traj)
mda_view

### 1.3 Implementation of the Coulomb matrix descriptor

The Coulomb matrix is defined as (Rupp et al., Phys. Rev. Lett., 108, 058301, 2012): 

$$
M_{ij} = 
\begin{cases}
  0.5 Z_i^{2.4}  & \mathrm{for} \ i = j\\
  \frac{Z_i Z_j}{R_{ij}}  & \mathrm{for} \ i \neq j
\end{cases}
$$



where the diagonal elements describe the potential energy of isolated atoms and the off-diagonal elements represent the pair-wise repulsion between nuclei $i$ and $j$.

We implement the Coulomb matrix descriptor based on https://github.com/pythonpanda/coulomb_matrix/blob/master/smiles_to_sparkcsv.py.

In [ ]:
def periodicfunc(element):
    """
    A function to output atomic number for each element in the periodic table
    """
    f = open("pt.txt")
    atomicnum = [line.split()[1] for line in f if line.split()[0] == element]
    f.close()
    return int(atomicnum[0])

def coulombmat(file,dim):
    """
    This function takes in an xyz input file for a molecule, 
    and the number of atoms in the biggest possible molecule 
    to compute the corresponding Coulomb matrix 
    """
    xyzfile=open(file)
    xyzheader = int(xyzfile.readline())
    xyzfile.close()
    i=0 ; j=0    
    cij=np.zeros((dim,dim))
    chargearray = np.zeros((xyzheader,1))
    xyzmatrix = np.loadtxt(file,skiprows=2,usecols=[1,2,3])
    atominfoarray = np.loadtxt(file,skiprows=2,dtype=str,usecols=[0])
    chargearray = [periodicfunc(symbol)  for symbol in atominfoarray]
    
    for i in range(xyzheader):
        for j in range(xyzheader):
            if i == j:
                cij[i,j]=0.5*chargearray[i]**2.4   # Diagonal term described by Potential energy of isolated atom
            else:
                dist = np.linalg.norm(xyzmatrix[i,:] - xyzmatrix[j,:])              
                cij[i,j] = chargearray[i]*chargearray[j]/dist   #Pair-wise repulsion 
    return  cij

def matsort(xyzfile,dim):
    """
    Takes in a Coloumb matrix of (mxn) dimension and performs a rowwise sorting such that ||C(j,:)|| > ||C(j+1,:)||, J= 0,1,.......,(m-1)
    Finally returns a vectorized (m*n,1) column matrix .
    """   
    unsorted_mat = coulombmat(xyzfile,dim)
    summation = np.array([sum(x**2) for x in unsorted_mat])
    sorted_mat = unsorted_mat[np.argsort(summation)[::-1,],:]    
    return sorted_mat.ravel()

### 1.4 Kernel Ridge Regression

Kernel Ridge Regression is an approach to supervised learning in which we are inferring a relationship between input and output. In our case, the input will be a molecule represented by a vector $\mathbf{X}$ and the output atomization energy $y$. We can view KRR as a basis set expansion since we write the relationship $\mathbf{X} \rightarrow y$ as:

$$
y(\mathbf{X'}) = \sum_i^N c_i K(\mathbf{X'}, \mathbf{X}_i)
$$

where our basis functions $K$ (the kernel functions) are centered on each training point and the $c_i$ are regression coefficients that can be obtained by minimizing a cost function. If we write the equation above in matrix form 
$$
\mathbf{y} = \mathbf{K} \vec{c}
$$
the optimisation problem reads:

$$
\quad \text{arg min}_{\vec{c}}\left(\frac{1}{2} ||\mathbf{K} \vec{c} - \mathbf{y}^{\mathrm{ref}} ||^2_2 + \frac{\alpha}{2} \vec{c}^T \mathbf{K} \vec{c}\right).
$$

This has the solution

$$
\quad \vec{c} = (\mathbf{K} + \alpha \mathbb{1} )^{-1} \mathbf{y},
$$

where $\alpha$ is the regularization parameter which encourages smoother models.





We will use is the coefficient of determination as the metric of model performance or score:

$$
R^2 = \left(1 - \frac{u}{v}\right),
$$

where $u$ is the residual sum of squares; $u = \sum_i \left( y_{i, \text{True}} - y_{\text{i, Pred}} \right) $ and $v$ is the total sum of squares: $v = \sum_i \left( y_{i, \text{True}} - \overline{y}_{\text{True}}\right)$. $ \overline{y}_{\text{True}}$ is the overall mean of our observed energies. The best score possibe is 1 (since $u$ vanishes) and it can get below $0$. An example of a model with an $R^2$ of $0$ is a constant model that always predicts $\overline{y}_{\text{True}}$.

Firstly, we will calculate the Coulomb matrix for each structure. Then, we will split our data into training and testing sets. This step could take a few minutes. During that continue reading.

In [ ]:
# 1. Define X (Coulomb matrices) and Y (energy)
all_CM = []
for i in inputfiles:
    CM = matsort(i,23)
    all_CM.append(CM)
all_CM = np.array(all_CM)
X = all_CM
Y = Energy

# 2. Split X and Y into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

# 3. Scale training and test set wrt. to the training set
X_mean = np.mean(X_train)
X_std  = np.std(X_train)
X_train_scaled = (X_train - X_mean)/X_std
X_test_scaled = (X_test - X_mean)/X_std
X_scaled = np.array((X-X_mean)/X_std)

# 4. Define 5-fold cross validation
rkf=RepeatedKFold(n_splits=5, n_repeats=1,  random_state=42)

We have prepared data and defined cross-validation. Have a look at the implementation of Kernel Ridge Regression using a linear kernel with 5-fold cross-validation. The optimal hyperparameter we are trying to find is the regularization parameter alpha for the linear kernel.

In [ ]:
print("LINEAR KERNEL")
print("Choice of alpha:")
all_scores={}
all_var={}

# Alpha in SciKit learn is the regularization parameter.
alphas = [ 1e-4, 1e-3, 1e-2, 0.1, 1, 5]
for alpha in alphas:
    print("alpha = ",alpha)
    score = []
    for train_index, test_index in rkf.split(X_train,Y_train):
        #We split the training set from before again in a training and validation set for each model.
        ##ttrain=traintrain, val=validation
        X_ttrain, X_val = X_train_scaled[train_index], X_train_scaled[test_index]
        Y_ttrain, Y_val = Y_train[train_index], Y_train[test_index]
        model = KernelRidge(kernel ='linear', alpha=alpha)
        model.fit(X_ttrain,Y_ttrain)
        Y_pred_val= model.predict(X_val)
        #The coefficient R^2 is defined as (1 - u/v), where 
        ##u is the residual sum of squares ((y_true - y_pred) ** 2).sum() and
        ##v is the total sum of squares ((y_true - y_true.mean()) ** 2).sum(). 
        score.append(model.score(X_val, Y_val))
    print ("Scores: ", score)
    print ("Average over cross validation: ", np.mean(score))
    print ("Variance: ", np.var(score))
    print ("----------------\n")
    all_scores.update({alpha:np.mean(score)})
    all_var.update({alpha:np.var(score)})
print ("All average scores: ", all_scores)
print ("All variances: ", all_var, "\n")
print ("Best average score: ", max(all_scores, key=all_scores.get))
print ("Best variance: ", min(all_var, key=all_var.get))

**Task 1:** Let us try out different kernel types and vary their parameters. Starting with a rough grid, we can check the performance at a fixed hyperparameter and then shrink the intervals to find optimal parameters.

The Radial-basis function kernel (rbf) kernel is given by:
$$
K(\mathbf{X}, (\mathbf{Y}) = exp\left( - \gamma || \mathbf{X} - \mathbf{Y} ||^2 \right)
$$
and the polynomial (poly) kernel by:
$$
K(\mathbf{X}, (\mathbf{Y}) = \left( \gamma \langle \mathbf{X} , \mathbf{Y} \rangle + 1 \right)^{3}
$$

In [ ]:
#These kernel types require more parameters, instead of doing a for-for loop let us import grid search from sklearn
from sklearn.model_selection import GridSearchCV

####################################
#### YOUR TURN: try the 'rbf' and 'poly' kernel and vary the hyperparameters
#define the model
model = KernelRidge(kernel ='')
    
# define the hyperparameters (alpha, gamma) that should be investigated in the grid search
# and choose their values. np.logspace(a, b, n) distributes n parameters logarithmically 
# between 10**a to 10**b.
params = {'alpha' : np.logspace(-6, 4, 5),
          'gamma': np.logspace(-6, 1, 5)
         }
####################################

#define the grid search with cross validation included
grid_search = GridSearchCV(estimator = model, #the estimator is your KRR model
                        param_grid = params, #the parameters to vary
                        scoring = 'r2', #R^2 scoring function
                        cv = 3, #3-fold cross-validation
                        verbose = 1, #more information in output
                        n_jobs = -1, #number of cores used, -1 uses all available
                        return_train_score=True)
#fit the model
grid_search.fit(X_train_scaled, Y_train)


#We plot the performance of all hyperparameters
pivot = pd.pivot_table(pd.DataFrame(grid_search.cv_results_),
                       index='param_gamma', 
                       columns='param_alpha', 
                       values='mean_test_score')

plt.subplots(figsize=(8,8))
sns.heatmap(pivot, cbar_kws={'label': 'mean $R^2$ score'})


# extract best estimator
print("Best estimator: ", grid_search.best_estimator_)
# extract score of best estimator
print("Best score: ", grid_search.best_score_)

**Task 2:** Test the best model on the test set (X_test_scaled, Y_test) and plot your predicted energies against the real ones in a scatter plot. 

In [ ]:
# We will compare training and prediction speeds:
import time

####################################
####YOUR TURN: try your best model, here is a bad one:
#define the model
model = KernelRidge(kernel = "sigmoid", alpha = 0.4, gamma = 0.0001)

####################################

#Fit the model and predict energies
start = time.time()
model.fit(X_train_scaled,Y_train)
end = time.time()
print("Wall-clock time for training:", end-start)

start = time.time()
Y_prediction= model.predict(X_test_scaled)
end = time.time()
print("Wall-clock time for predicting:", end-start)

print("MAE: ", np.mean(np.abs(Y_prediction-Y_test)), "kcal/mol")
print("MSE: ", np.mean(np.abs(Y_prediction-Y_test)**2))
print("Score: ", model.score(X_test_scaled, Y_test))

#Plot result
plt.figure()
plt.scatter(Y_test,Y_prediction)
a = np.linspace(-2500,0)
plt.plot(a,a,color="r")
plt.xlabel("E(test) [kcal/mol]")
plt.ylabel("E(prediction) [kcal/mol]")
plt.show()

### 1.5 Support Vector Regression

In support vector regression, samples lying outside the $\epsilon$-tube are penalized by their distance to the margin boundary: $\zeta_i , \zeta_i^* \geq 0$, depending on whether they lie above or below the tube. The penalty has the form $C \sum_{i=1}^n (\zeta_i + \zeta_i^*)$, thus $C$ acts as an inverse regularization parameter.

A grid search for support vector regression takes on a similar shape, but with the tubewidth $\epsilon$ and the penalizations strength $C$. In contrast to KRR, fitting a SVR model can not be done in closed-form and training is typically slower. On the other hand, since SVR learns a sparse model (in contrast to KRR) one expects the prediction to be faster (for large models). The additional arithmetic operations required in KRR may however be compensated by computational details in how the kernel functions are computed.

**Task 3:** Let us try out different kernel types and vary their parameters.

In [ ]:
#import grid search from sklearn
from sklearn.model_selection import GridSearchCV

####################################
####YOUR TURN: try the 'rbf' and 'poly' kernel and vary the hyperparameters
#define the model
model = SVR(kernel ='')

#define the hyperparameters (C, epsilon) that should be investigated in the grid search
#and choose their values 
#default: gamma='scale'
params = {'C' : np.logspace(-1, 4, 5),
          'epsilon': np.logspace(-1, 2, 5)
         }
####################################


#define the grid search with cross validation included
grid_search = GridSearchCV(estimator = model, #the estimator is your KRR model
                        param_grid = params, #the parameters to vary
                        scoring = 'r2', #R^2 scoring function
                        cv = 3, #3-fold cross-validation
                        verbose = 1, #more information in output
                        n_jobs = -1, #number of cores used, -1 uses all available
                        return_train_score=True)
#fit the model
grid_search.fit(X_train_scaled, Y_train.ravel())


#We plot the performance of all hyperparameters
pivot = pd.pivot_table(pd.DataFrame(grid_search.cv_results_),
                       index='param_C', 
                       columns='param_epsilon', 
                       values='mean_test_score')

plt.subplots(figsize=(8,8))
sns.heatmap(pivot, cbar_kws={'label': 'mean $R^2$ score'})

# extract best estimator
print("Best estimator: ", grid_search.best_estimator_)
# extract score of best estimator
print("Best score: ", grid_search.best_score_)

With the rbf kernel we found:<br> 
Best estimator:  SVR(C=10000.0, epsilon=0.6812920690579614)<br> 
With score:  0.9900406583028621

With the poly kernel we found:<br>
Best estimator:  SVR(C=100000.0, epsilon=35.93813663804626, kernel='poly')<br>
With score:  0.9378256191280022

Have you managed to find a better model?

**Task 4:** Test the kernel on the test set (X_test_scaled, Y_test) and plot your predicted energies against the real ones in a scatter plot. How does the wall time for training compare to KRR? How about prediction time?

In [ ]:
gamma='scale' #default

####################################
#### YOUR TURN: try your best model, here is a bad one:
#define the model 
model = SVR(kernel ='sigmoid', epsilon=0.111, C=1214, gamma=gamma)
####################################

#Fit the model and predict energies
start = time.time()
model.fit(X_train_scaled,Y_train)
end = time.time()
print("Wall-clock time for training:", end-start)

start = time.time()
Y_prediction= model.predict(X_test_scaled)
end = time.time()
print("Wall-clock time for predicting:", end-start)

train_size = len(Y_train)
sv_ratio = model.support_.shape[0] / train_size
print("Support vector ratio: %.3f" % sv_ratio)

print("MAE: ", np.mean(np.abs(Y_prediction-Y_test)), "kcal/mol")
print("MSE: ", np.mean(np.abs(Y_prediction-Y_test)**2))
print("Score: ", model.score(X_test_scaled, Y_test))

plt.figure()
plt.scatter(Y_test,Y_prediction)
a = np.linspace(-2500,0)
plt.plot(a,a,color="r")
plt.xlabel("E(test) [kcal/mol]")
plt.ylabel("E(prediction [kcal/mol])")
plt.show()